# NAVIGATOR 

Navigator is a proposal for an RL agent and a customized environment inspired by work we did in both AIMEE and REMATH. 


### minimalist approach / toy example

Reinforcement learning is designed to work without a lot of priors or explicit feature engineering. The most important thing is how the problem is modeled. A hybrid system that uses an RL/GA based approach to explore code trajectories to characterize program execution would have the following characteristics:

action space: this would be any possible single instruction input to the program programmatically or by a human operator, in REMATH this consisted of the set of all possible instructions given the number of registers, number of writeable registers and possible operations in the function set. "PalmTree: Learning an Assembly Language Model for Instruction Embedding" has an interesting discussion of three different methods for representing instructions: one-hot encoding, feature engineering, and dense vector approaches à la Bert/word2vec

state: state would be any way of modeling programmatic state, state transitions would be defined as consiisting of an action from the action space that transitions $S_t$ to $S_{t+1}

reward: how do you characterize a "successful" operation and how can this be guided by a human operator. 

REINFORCE- the most basic policy gradient algorithm- uses Monte Carlo methods to estimate return using samples of episodes to update policy parameters. This works because of the policy gradient theorem, namely that the expectation of a sample of the gradient is equivalent to the actual gradient

$$ J(\theta) = \mathbb{E}_{\pi} [ G_t \triangledown ln \pi_\theta(A_t | S_t)] $$

in order to reach an estimate of $J_\theta$ the following process is followed until convergence criteria is met 
1. intiialize the policy parameter $\theta$ at random 
2. generate a single trajectory (in our case this would be a single linear program) $\pi_{\theta} : S_{1}, A_{1}, R_{2}, S_{2}, A_{2}....S_{T} $
3. For t=1....T
    1. estimate the return G_t
    2. update the policy parameters  $\theta <- \theta + \alpha\lambda^tG_t\triangledown_\theta ln \pi_\theta(A_t | S_t)] $
    
### optimized example 
Proximal Policy Optimization (PPO) combines trust-region techniques with policy gradient methods like REINFORCE. The key innovation of PPO is its alternation between sampling data through direct environment interaction and optimizing a surrogate objective function using stochastic gradient ascent. This kind of architecture is called an actor-critic architecture since it learns both the policy (actor) and the value-function (critic). It does this mainly to reduce gradience variance caused by variance in the estimate 

**critic** updates the value function parameters (in our case it would likely be action-value $Q_w(a|s)$ although it could be state-value)  
**actor** updates the p9olicy parameters for $\theta$ for $\pi_\theta(a|s)$

In step 3.2. of REINFORCE you could compute a probability ratio between old and new policies
$$r(\theta) = \frac{\pi_\theta(a|s)}{\pi_{\theta_{old}}(a|s)}$$

$$J^{CLIP}(\theta) = \mathbb{E}[min(r\theta)A_theta(s,a), clip(r(\theta), 1- \epsilon, 1 + \epsilon) A_\theta(s,a$$

this bounds the maximum difference between the old and new policies to avoid parameter updates that change the policy too much at one step. 

since our action space is discrete with sparse high rewards (imagine normal code exeuction as a bimodal distribution with much code having no reward, some code having medium rewaard, and in the context of a reward function designed for emergent execution behavior very little code having high reward. The regularization mode above referred to as KL regularizzation helps resolve this failure mode 

This is a bit complicated since we are actually most interested in those trajectories that change the policy a great deal as those represent the high-reward/high-risk trajectories likely to be emergent execution attacks. We can balance our need for both training stability that uses KL-divergence ("curiosity metric") to avoid local optima with a risk-aware method of training the policy that optimizes those trajectories that result in best-case performance (at the expense of average case performance) as the task of **NAVIGATOR** is somewhere between RL and anomaly detection

### terminology

$\pi(a|s)$ policy or agent behavior strategy which can be parameterized by $\theta$ which is in turn parameterized by training on batches of episodes or via the introduction of constraints 

**state value function** the e
expected return of state $s; V_{w}(.)$ value function parameterized by w

**action value function** similar to $V(s)$ but asseses the expected return of a pair of state and action $(s, a) Q_w(.( an action value function parameterized by w

### actor/critic

The actor model is estimates the policy $\pi(a|s)$ and the critic model estimates the $Q(s, a)$ or action value function. Since programmatic instructions are intrinsically sequential both of these models are modeled as recurrent-type language models. There is a great deal of experimentation still needed in this regarded i.e. how does accuracy degrade from using a more complex language model like an LSTM vs a simple RNN, what metric can be designed to characterize the complexity of the context window sent to the sequence model etc. 

### reward

In policy gradient learning a reward is a scalar assigned to a batch of trajectories computed according to a policy, it is subsequently used to update the policy parameters. Rewards can be layered and can either be extrinsic or intrinsic or both. Vanilla PPO has an intrinsic reward consisting of the KL divergence between the previous policy and the current policy. Another popular method of framing intrinsic reward is curiosity.

### constraints

Constraints can be modeled in a number of ways and there are two primary candidates for how they might be modeled in NAVIGATOR, in situ constraints that reject linear programs as they are generated by the policy network or constraint-based genetic programming that would operate on a policy embedding. 